#### Generate diagram of shear zone property

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
TwoDSubduction_DIR = os.environ['TwoDSubduction_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
from shilofue.TwoDSubduction0.Group import CASE_SUMMARY, PlotGroupDiagram
import shilofue.TwoDSubduction0.VtkPp as TwoDVtkPp
from shilofue.Group import ReadBasicInfoGroup
from shilofue.TwoDSubduction0.PlotVisit import VISIT_OPTIONS

In [ ]:
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT17') # different ages
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT18') # different sz properties
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT20') # different ages
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT21') # different ages
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT23_4') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT23_6') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_0') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_1') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_2') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_3') # different sz viscosity, depth, width
# group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_4') # different sz viscosity, depth, width
group_dir = os.path.join(TwoDSubduction_DIR, 'EBA_CDPT24_5') # different sz viscosity, depth, width
assert(os.path.isdir(group_dir))

#### Generate the slab morphology at the shortest interval possible

First, run the following block.

Then, take the py_temp.sh file generated and:

    chmod +x py_temp.sh
    
    ./py_temp.sh

This would run in parallel and generate the slab_morph.txt files.

In [ ]:
time_interval = 0.1e6

# read the list of cases
case_list, _, _, _ = ReadBasicInfoGroup(group_dir)

# write a .sh file to run command in the system
py_temp_file = os.path.join(ASPECT_LAB_DIR, 'py_temp.sh')
py_commands = []

for _case in case_list:
  case_dir = os.path.join(group_dir, _case)
  py_command = "python -m shilofue.TwoDSubduction0.VtkPp morph_case_parallel -i %s -ti %.2e\n" % (case_dir, time_interval)
  py_commands.append(py_command)

with open(py_temp_file, 'w') as fout:
  fout.write("#!/bin/bash\n")
  for py_command in py_commands:
    fout.write(py_command)

#### Write to a summary file and generates plot for publication

In [ ]:
from matplotlib import get_configdir

get_configdir()

In [ ]:
plt.style.use('publication_lhy11009')


all_actions = ['t660', "t800", "t1000", 'shear_zone', "strength", "sd_modes", "Vage", "ages"] # all actions included in summary
t1_method = 't660' # method for the range of time, t660: multiply the t660 to get a range
t1_factor = 5.0 # if t1_method = 't660', then this is the factor to multiply

assert(os.path.isdir(group_dir))

# defaults
write_for_paper = False; update_from_group = False

# if we want to update the information on cases in a group, set update_from_group to True 
append = ""
# Total summary, don't update
# o_path_csv = os.path.join(TwoDSubduction_DIR, 'case_summary.csv'); update_from_group = False
# update from the group
# o_path_csv = os.path.join(group_dir, 'case_summary.csv'); update_from_group = True 
# for export to a latex table for the paper
write_for_paper = True; update_from_group = False; o_path_csv = os.path.join(TwoDSubduction_DIR, 'case_summary.csv'); o_path_tex = os.path.join(TwoDSubduction_DIR, 'case_summary.tex')

# o_path = os.path.join(group_dir, 'case_summary_sz_visc.txt'); update_from_group = False # shear zone, visc = 1e20
# o_path = os.path.join(group_dir, 'case_summary_sz_visc_1e19.txt'); append=19; update_from_group = False # shear zone, visc = 1e19
# o_path = os.path.join(group_dir, 'case_summary_sz_visc_1e21.txt'); append=21; update_from_group = False # shear zone, visc = 1e19
# o_path = os.path.join(group_dir, 'case_summary_ages.txt'); update_from_group = False
# o_path = os.path.join(group_dir, 'case_summary_SA40_OA20.txt')
# o_path = os.path.join(group_dir, 'case_summary_SA140_OA40.txt')
    
img_dir = os.path.join(group_dir, 'img')
if not os.path.isdir(img_dir):
  os.mkdir(img_dir)

fig_vtr_path = os.path.join(img_dir, 'vtr_pub%s.pdf' % append)
fig_vage_path = os.path.join(img_dir, 'vag_pub%s.pdf' % append)
# fig_vtr_path = os.path.join(img_dir, 'vtr_pub_SA40_OA20.pdf')
# fig_vtr_path = os.path.join(img_dir, 'vtr_pub_SA140_OA40.pdf')

Case_Summary = CASE_SUMMARY(VISIT_OPTIONS=VISIT_OPTIONS)

# import old result if it exists 
if os.path.isfile(o_path_csv):
  Case_Summary.import_file(o_path_csv)
  Case_Summary.Update(actions=all_actions, t1_method=t1_method, t1_factor=t1_factor)

# import the new directory
if update_from_group:
  Case_Summary.import_directory(group_dir, actions=all_actions, t1_method=t1_method, t1_factor=t1_factor)



# output file output 
if write_for_paper: 
  Case_Summary.write_file_for_sz_paper(o_path_tex)
else:
  Case_Summary.write_file(o_path_csv)

#### plot options
# plot the velocity - shear zone properties diagram: trench
# Case_Summary.plot_velocities_tr_publication(fig_vtr_path)

# plot the velocity - ages diagram
# Case_Summary.plot_velocities_ages_publication(fig_vage_path)

# resume default configurations
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)